## Recipe Builder Actions Overview

### Saving a File Cell
If you wish to save the contents of a cell, simply run it.  The `%%writefile` command at the top of the cell will write the contents of the cell to the file named at the top of the cell. You should run the cells manually when applicable. However, **pressing any of the actions at the top will automatically run all file cells relevant to the action**.

### Training and Scoring
Press the associated buttons at the top in order to run training or scoring. The training output will be shown below the `evaluator.py` cell and scoring output will be shown below the `datasaver.py` cell. You must run training at least once before you can run scoring. You may delete the output cell(s). Running training the first time or after changing `requirements.txt` will be slower since the dependencies for the recipe need to be installed, but subsequent runs will be significantly faster. If you wish to see the hidden output add `debug` to the end of the output cell and re-run it.

### Creating the Recipe
When you are done editing the recipe and satisfied with the training/scoring output, you can create a recipe from the notebook by pressing `Create Recipe`. You must run scoring at least once before you can create the recipe. After pressing it, you will see a progress bar showing how much time is left for the build to finish. If the recipe creation is successful the progress bar will be replaced by an external link that you can click to navigate to the created recipe.


## Caution!
* **Do not delete any of the file cells**
* **Do not edit the `%%writefile` line at the top of the file cells**

---

#### **Requirements File** (Optional)
Add additional libraries you wish to use in the recipe to the cell below. You can specify the version number if necessary. The file cell below is a **commented out example**.  The file structure is yaml. It adheres to the specification outlined in the [Conda Documentation](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-file-manually). **NOTE:** The name field is not allowed to be overridden in the file.

In [ ]:
%%writefile ~/my-workspace/.recipes/recipe-cyiuzaLij/requirements.txt

#dependencies:
## Conda dependencies should be attempted first
#- statsmodels=0.10.2
#- pip
#- pip:
## Pip installs can be listed next but should only be used when Conda is unavailable
#   - pmdarima==1.5.3


Search here for additional libraries https://anaconda.org/. This is the list of main **libraries already in use**:
`python=3.6.7` `scikit-learn` `pandas` `numpy` `data_access_sdk_python`
**Warning: libraries or specific versions you add may be incompatible with the above libraries**.

---

#### **Configuration Files**
List any hyperparameters you wish to use. Specify the dataset(s) and schema(s) that are needed for training/scoring. To find the dataset ids go to the **Data tab** in Adobe Experience Platform or view the **Datasets** folder in the **Notebooks Data tab** on the left. You can also find schema id in the **Notebooks Data tab** under the **Schemas** folder. Each configuration will only be used for its corresponding action. `ACP_DSW_TRAINING_XDM_SCHEMA` and `ACP_DSW_SCORING_RESULTS_XDM_SCHEMA` will only be used after the recipe has been created.

##### Training Configuration

In [ ]:
%%writefile ~/my-workspace/.recipes/recipe-cyiuzaLij/training.conf

{
   "trainingDataSetId": "605aa0b37d909c194873348a",
   "ACP_DSW_TRAINING_XDM_SCHEMA": "https://ns.adobe.com/trowelab/schemas/c9ed4fec408542746fbc0de1e2f1ebd1cc6fbe378985f3b3",
   "tenantId": "_trowelab", 
   "max_iter": "50000"
}

##### Scoring Configuration

In [ ]:
%%writefile ~/my-workspace/.recipes/recipe-cyiuzaLij/scoring.conf

{
   "scoringDataSetId": "605aa0eae30ac41949d41637",
   "scoringResultsDataSetId": "605aa30ec251b71948f2dab4",
   "ACP_DSW_SCORING_RESULTS_XDM_SCHEMA": "https://ns.adobe.com/trowelab/schemas/23e6fba6f38b1aea21502439fba5695916c4632e511ecdf4",
   "tenantId": "_trowelab"
}

**The following configuration parameters are automatically set for you when you train/score:** 
`ML_FRAMEWORK_IMS_USER_CLIENT_ID` `ML_FRAMEWORK_IMS_TOKEN` `ML_FRAMEWORK_IMS_ML_TOKEN` `ML_FRAMEWORK_IMS_TENANT_ID`

---

#### **Training Data Loader File**
Implement the `load` function to load and prepare the training data.

In [ ]:
%%writefile ~/my-workspace/.recipes/recipe-cyiuzaLij/recipe/trainingdataloader.py

import pandas as pd
from datetime import datetime, timedelta
from platform_sdk.dataset_reader import DatasetReader
from .utils import get_client_context

def load(config_properties):
    print("Training Data Load Start")

    #########################################
    # Load Data
    #########################################    
    client_context = get_client_context(config_properties)
    
    dataset_reader = DatasetReader(client_context, config_properties['trainingDataSetId'])
    
    tenant_id = config_properties.get("tenant_id")
    
    dataframe = dataset_reader.read()

    #########################################
    # Data Preparation/Feature Engineering
    #########################################    
    dataframe = dataframe.drop("UserID", axis=1)
    dataframe = dataframe.astype('int32')
    dataframe = pd.concat([dataframe["checked_delivery_detail"], dataframe["saw_checkout"], dataframe["sign_in"], dataframe["ordered"]], axis=1)

    print("Training Data Load Finish")
    return dataframe

---

#### **Scoring Data Loader File**
Implement the `load` function to load and prepare the scoring data.

In [ ]:
%%writefile ~/my-workspace/.recipes/recipe-cyiuzaLij/recipe/scoringdataloader.py

import pandas as pd
from datetime import datetime, timedelta
from .utils import get_client_context
from platform_sdk.dataset_reader import DatasetReader

def load(config_properties):

    print("Scoring Data Load Start")

    #########################################
    # Load Data
    #########################################
    client_context = get_client_context(config_properties)

    dataset_reader = DatasetReader(client_context, config_properties['scoringDataSetId'])
    tenant_id = config_properties.get("tenant_id")

    dataframe = dataset_reader.read()

    #########################################
    # Data Preparation/Feature Engineering
    #########################################

    dataframe = pd.concat([dataframe["UserID"], dataframe["checked_delivery_detail"].astype('int32'), dataframe["saw_checkout"].astype('int32'), dataframe["sign_in"].astype('int32')], axis=1)
    
    print("Scoring Data Load Finish")

    return dataframe

---

#### **Pipeline File**
Implement the `train` function and return the trained model. Implement the `score` function to return a prediction made on the scoring data.

In [ ]:
%%writefile ~/my-workspace/.recipes/recipe-cyiuzaLij/recipe/pipeline.py

from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor

def train(config_properties, data):

    print("Train Start")
        
    #########################################
    # Extract fields from configProperties
    #########################################
    max_iter = int(config_properties['max_iter'])


    #########################################
    # Fit model
    #########################################
    X_train = data.drop('ordered', axis=1).values
    y_train = data['ordered']

    model = svm.SVC(probability=True, max_iter=max_iter).fit(X_train, y_train)

    print("Train Complete")
    return model

def score(config_properties, data, model):

    print("Score Start")

    X_test = data.drop('UserID', axis=1).values
    y_pred = model.predict(X_test)

    data['prediction'] = y_pred
    #data = data[['UserID', 'prediction']].reset_index()

    print("Score Complete")
    return data

---

#### **Evaluator File**
Implement the `split` function to partition the training data and the `evaluate` function to the return the validation metrics you wish to see. Training output will be shown below this file cell.

In [ ]:
%%writefile ~/my-workspace/.recipes/recipe-cyiuzaLij/recipe/evaluator.py

from sklearn.metrics import precision_score, recall_score, roc_curve, confusion_matrix
import numpy as np

class Evaluator():
    def __init__(self):
        print ("Initiate")

    def split(self, config={}, dataframe=None):
        
        train = dataframe[0:5000]
        val = dataframe[5000:]

        return train, val

    def evaluate(self, data=[], model={}, config={}):
        print ("Evaluation triggered")

        val = data.drop('ordered', axis=1)
        y_pred = model.predict(val)
        y_actual = data['ordered'].values
             
        acc = model.score(val, y_actual)
        recall = recall_score(y_actual, y_pred)
        precision = precision_score(y_actual, y_pred)

        metric = [{"name": "Accuracy", "value": acc, "valueType": "double"},
                  {"name": "Precision", "value": precision, "valueType": "double"},
                  {"name": "Recall", "value": recall, "valueType": "double"}]
        
        print(metric)
        return metric

In [ ]:
!bash -e run_action.sh recipe-cyiuzaLij training

---

#### **Data Saver File**
Implement the `save` function for saving your prediction. Scoring output will be added below this cell.

In [ ]:
%%writefile ~/my-workspace/.recipes/recipe-cyiuzaLij/recipe/datasaver.py

import pandas as pd
from .utils import get_client_context
from platform_sdk.models import Dataset
from platform_sdk.dataset_writer import DatasetWriter

def save(config_properties, prediction):
  print("Datasaver Start")

  client_context = get_client_context(config_properties)
  tenant_id = config_properties.get("tenantId")

  dataset = Dataset(client_context).get_by_id(config_properties['scoringResultsDataSetId'])
  dataset_writer = DatasetWriter(client_context, dataset)
  dataset_writer.write(prediction, file_format='json')

  print("Datasaver Finish")

In [ ]:
!bash -e run_action.sh recipe-cyiuzaLij scoring